# 在逆向 SIMPACK 的曲线线路上，绘制整车各组件 DoFs 随时间的变化

## 轨道

轨道本身的三维位置和朝向存储在 trajectory_data.npz 中：

$s\_vals,\; xvals,\; yvals,\; zvals,\; \psi\_vals,\; (\phi\_vals)$

其中：
- $s\_vals$ 表示离散的里程点
- $\{xvals, yvals, zvals\}$ 是轨道在全局系的平移位置
- $\psi\_vals$、$\phi\_vals$ 是轨道在全局系的旋转角度

## 车辆部件(轮对、转向架、车体)

它们相对于"轨道系"下仅有5个自由度(2个平移 + 3个转角)，因为在轨道方向的偏移由里程 $s_{query}$ 已经管理

具体来说：
- 局部坐标中的 $(x_{\mathrm{local}}, y_{\mathrm{local}}, z_{\mathrm{local}})$ 可简化为 $(0.0,\; y_{\mathrm{local}},\; z_{\mathrm{local}})$
- 旋转部分 $(\mathrm{roll}_{\mathrm{local}},\,\mathrm{pitch}_{\mathrm{local}},\,\mathrm{yaw}_{\mathrm{local}})$ 依旧保留3个旋转自由度

## 数据结构

在运行时，每个车辆组件由**6个浮点数**定义：

1. $s_{\text{query}}$ — 轨道里程（1个）
2. $(y_{\mathrm{local}}, z_{\mathrm{local}}, \mathrm{roll}_{\mathrm{local}}, \mathrm{pitch}_{\mathrm{local}}, \mathrm{yaw}_{\mathrm{local}})$ — 该部件相对于轨道系的5个自由度


## 坐标转换流程

1. 通过 $s_{\text{query}}$ 在轨道数据中插值，得到轨道系在全局坐标系的位置和朝向
2. 将部件的局部5自由度与轨道系位姿结合，计算部件在全局坐标系下的完整6D位姿
3. 转换过程通常使用齐次变换矩阵实现，将旋转和平移操作统一处理

In [ ]:
%matplotlib widget

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import json

from tools_RailTransform import (
    # 轨道姿态获取
    get_track_pose,
    # 坐标变换
    make_transform, transform_points_3d,
    # 几何体建模
    create_cylinder_mesh, create_box_mesh,
    # 绘制辅助
    draw_wheelset, draw_box,
    # 读SIMPACK文件
    read_simpack_6dof
)


# ========== 1) 轨道中心线数据 ==========
try:
    with open('trajectory_data.json', 'r') as file:
        trajectory_data = json.load(file)

    # 获取轨道数据
    s_vals   = np.array(trajectory_data.get('s', []))
    xvals    = np.array(trajectory_data.get('x', []))
    yvals    = np.array(trajectory_data.get('y', []))
    zvals    = np.array(trajectory_data.get('z', []))
    psi_vals = np.array(trajectory_data.get('psi', []))
    phi_vals = np.array(trajectory_data.get('phi', []))
    left_rail  = np.array(trajectory_data.get('left_rail', []))
    right_rail = np.array(trajectory_data.get('right_rail', []))

    print(f"成功从JSON文件加载了{len(s_vals)}个轨道点数据")
except Exception as e:
    print(f"加载JSON文件时出错: {e}")
    # 初始化为空数组以避免后续代码报错
    s_vals = xvals = yvals = zvals = psi_vals = phi_vals = np.array([])
    left_rail = right_rail = np.array([]).reshape(0, 3)


# ========== 2) 准备几何网格 ==========
# 生成车轴网格
axle_length   = 2.0
axle_radius   = 0.065
X_axle, Y_axle, Z_axle = create_cylinder_mesh(axle_length, axle_radius)
pts_axle = np.column_stack([X_axle.ravel(), Y_axle.ravel(), Z_axle.ravel()])

# 生成车轮网格
wheel_radius  = 0.43
wheel_thickness = 0.04
wheel_offset  = 0.7175
X_wh, Y_wh, Z_wh = create_cylinder_mesh(wheel_thickness, wheel_radius)
pts_wheel = np.column_stack([X_wh.ravel(), Y_wh.ravel(), Z_wh.ravel()])

# 车体box
car_body_faces  = create_box_mesh(L=25.0, W=3.0, H=3.0)
# 转向架box
bogie_box_faces = create_box_mesh(L=3.0,  W=2.5, H=0.5)


# ========== 3) 读取 SIMPACK 文件 ==========
file_path = os.path.join('..', 'PostAnalysis', 'Result_Y_RosRt.log')
df_wheel = read_simpack_6dof(file_path)
print("SIMPACK文件读取完毕, 行数:", len(df_wheel))

# ========== 4) 定义几何网格 (车轴、车轮、车体、转向架) ==========
# 车轴
axle_length   = 2.0
axle_radius   = 0.065
X_axle, Y_axle, Z_axle = create_cylinder_mesh(axle_length, axle_radius)
pts_axle = np.column_stack([X_axle.ravel(), Y_axle.ravel(), Z_axle.ravel()])

# 车轮
wheel_radius  = 0.43
wheel_thickness = 0.04
wheel_offset  = 0.7175
X_wh, Y_wh, Z_wh = create_cylinder_mesh(wheel_thickness, wheel_radius)
pts_wheel = np.column_stack([X_wh.ravel(), Y_wh.ravel(), Z_wh.ravel()])

# 车体 (大盒子)
car_body_faces  = create_box_mesh(L=25.0, W=3.0, H=3.0)

# 转向架 (小盒子)
bogie_box_faces = create_box_mesh(L=3.0,  W=2.5, H=0.4)

# ========== 5) 主绘图 (包含轮对、车体等) ==========
fig = plt.figure(figsize=(14,6))
ax = fig.add_subplot(111, projection='3d')

# 先画轨道线
ax.plot(xvals, yvals, zvals, color='black', alpha=0.4, label='Track Center')
ax.plot(left_rail[:,0], left_rail[:,1], left_rail[:,2], 'r-', alpha=0.4, label='Left Rail')
ax.plot(right_rail[:,0], right_rail[:,1], right_rail[:,2], 'b-', alpha=0.4, label='Right Rail')

# 选取某一帧 (例如 第555行)
row_idx = 555
row = df_wheel.iloc[row_idx]

# (A) 绘制 4 个轮对
for ws_id in [1,2,3,4]:
    s_  = row[f'y_ws{ws_id:02d}_x']
    y_  = row[f'y_ws{ws_id:02d}_y']
    z_  = row[f'y_ws{ws_id:02d}_z']
    r_  = row[f'y_ws{ws_id:02d}_roll']
    p_  = row[f'y_ws{ws_id:02d}_pitch']
    yw_ = row[f'y_ws{ws_id:02d}_yaw']

    # 轨道->全局 变换
    X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(s_, s_vals, xvals, yvals, zvals, psi_vals, phi_vals)
    T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)

    # 轮对->轨道
    T_W2T = make_transform(yw_, p_, r_, 0.0, y_, z_)

    # 合成: 轮对->全局
    T_W2G = T_T2G @ T_W2T

    draw_wheelset(ax, T_W2G,
                  pts_axle, X_axle.shape,
                  pts_wheel, X_wh.shape,
                  wheel_offset,
                  color_axle='magenta',
                  color_wheel='cyan',
                  alpha=1)

# (B) 绘制转向架 (示例 2个)
for bg_id in [1,2]:
    s_bg  = row[f'y_f0{bg_id}_x']
    y_bg  = row[f'y_f0{bg_id}_y']
    z_bg  = row[f'y_f0{bg_id}_z']
    r_bg  = row[f'y_f0{bg_id}_roll']
    p_bg  = row[f'y_f0{bg_id}_pitch']
    yw_bg = row[f'y_f0{bg_id}_yaw']

    X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(s_bg, s_vals, xvals, yvals, zvals, psi_vals, phi_vals)
    T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)

    T_bg2T = make_transform(yw_bg, p_bg, r_bg, 0.0, y_bg, z_bg)
    T_bg2G = T_T2G @ T_bg2T

    draw_box(ax, T_bg2G, bogie_box_faces, color='green', alpha=0.3)

# (C) 绘制车体
# 特殊规定: 车体几何高度 1.8, 并需要翻转180°
s_cb   = row['y_cb_x']
y_cb   = row['y_cb_y']
z_cb   = row['y_cb_z'] - 1.8 # hc = -1.8
r_cb   = row['y_cb_roll']
p_cb   = row['y_cb_pitch']
yw_cb  = row['y_cb_yaw']

X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(s_cb, s_vals, xvals, yvals, zvals, psi_vals, phi_vals)
T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)
T_cb2T = make_transform(yw_cb, p_cb, r_cb, 0.0, y_cb, z_cb)
T_cb2G = T_T2G @ T_cb2T

# 车体需要翻转180°
T_flipCar = make_transform(0, 0, math.pi, 0, 0, 0)
T_cb2G = T_cb2G @ T_flipCar

draw_box(ax, T_cb2G, car_body_faces, color='orange', alpha=0.2)

# ========== 6) 坐标轴、比例、视图等设置 ==========

ax.set_xlabel("X [m]")
ax.set_ylabel("Y [m]")
ax.set_zlabel("Z [m]")
ax.set_title("Global View (Z Down), with Wheels/Bogies/Carbody")
ax.legend()

# 强制等比例显示
x_min, x_max = xvals.min(), xvals.max()
y_min, y_max = yvals.min(), yvals.max()
z_min, z_max = zvals.min(), zvals.max()
cx = 0.5 * (x_min + x_max)
cy = 0.5 * (y_min + y_max)
cz = 0.5 * (z_min + z_max)
max_range = max((x_max - x_min), (y_max - y_min), (z_max - z_min)) * 0.5
ax.set_xlim(cx - max_range, cx + max_range)
ax.set_ylim(cy - max_range, cy + max_range)
ax.set_zlim(cz - max_range, cz + max_range)
ax.set_box_aspect((1,1,1))

# 反转 Z 轴: 如果需要 z 越大在图中越低
ax.invert_zaxis()

# 调整视角
ax.view_init(elev=25, azim=70, roll=0)

plt.show()

In [ ]:
%matplotlib widget

import os
import json
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ========== (A) 从工具脚本中导入所需函数 ==========
from tools_RailTransform import (
    # 轨道姿态、坐标变换
    get_track_pose, make_transform, transform_points_3d,
    # 几何建模
    create_cylinder_mesh, create_box_mesh,
    # 绘制函数
    draw_box, draw_wheelset,
    # SIMPACK文件读取
    read_simpack_6dof
)

# ========== (B) 读取轨道数据 JSON ==========
try:
    with open('trajectory_data.json', 'r') as file:
        trajectory_data = json.load(file)

    s_vals   = np.array(trajectory_data.get('s', []))
    xvals    = np.array(trajectory_data.get('x', []))
    yvals    = np.array(trajectory_data.get('y', []))
    zvals    = np.array(trajectory_data.get('z', []))
    psi_vals = np.array(trajectory_data.get('psi', []))
    phi_vals = np.array(trajectory_data.get('phi', []))

    left_rail  = np.array(trajectory_data.get('left_rail', []))
    right_rail = np.array(trajectory_data.get('right_rail', []))

    print(f"[Info] 成功从JSON文件加载了{len(s_vals)}个轨道点")
except Exception as e:
    print(f"[Error] 加载JSON文件时出错: {e}")
    # 保证后面不出错
    s_vals = xvals = yvals = zvals = psi_vals = phi_vals = np.array([])
    left_rail = right_rail = np.array([]).reshape(0,3)

# ========== (C) 读取 SIMPACK 输出结果 ==========
file_path = os.path.join('..', 'PostAnalysis', 'Result_Y_RosRt.log')
df_wheel = read_simpack_6dof(file_path)
print("[Info] SIMPACK文件读取完毕, 行数 =", len(df_wheel))

# ========== (D) 定义几何网格 (车轴、车轮、车体、转向架) ==========
# 1) 车轴
axle_length   = 2.0
axle_radius   = 0.065
X_axle, Y_axle, Z_axle = create_cylinder_mesh(axle_length, axle_radius)
pts_axle = np.column_stack([X_axle.ravel(), Y_axle.ravel(), Z_axle.ravel()])

# 2) 车轮
wheel_radius  = 0.43
wheel_thickness = 0.04
wheel_offset  = 0.7175
X_wh, Y_wh, Z_wh = create_cylinder_mesh(wheel_thickness, wheel_radius)
pts_wheel = np.column_stack([X_wh.ravel(), Y_wh.ravel(), Z_wh.ravel()])

# 3) 车体
car_body_faces  = create_box_mesh(L=25.0, W=3.0, H=3.0)

# 4) 转向架
bogie_box_faces = create_box_mesh(L=3.0, W=2.5, H=0.4)

# ========== (F) 绘制与包围盒 ==========
# 这里假设选用最后一帧 row_idx = len(df_wheel) - 1
row_idx = len(df_wheel) - 1
rowF = df_wheel.iloc[row_idx]

fig2 = plt.figure(figsize=(8,6))
ax2 = fig2.add_subplot(projection='3d')

# (1) 先画全轨道
ax2.plot(xvals, yvals, zvals, color='black', alpha=0.8, label='Track Centerline')
ax2.plot(left_rail[:,0], left_rail[:,1], left_rail[:,2], 'r-', alpha=0.8, label='Left Rail')
ax2.plot(right_rail[:,0], right_rail[:,1], right_rail[:,2], 'b-', alpha=0.8, label='Right Rail')

# (2) 准备汇总所有部件的全局顶点
pts_all_in_G = []

# ============ (2A) 4 轮对 ============
for ws_id in [1,2,3,4]:
    s_  = rowF[f'y_ws{ws_id:02d}_x']
    y_  = rowF[f'y_ws{ws_id:02d}_y']
    z_  = rowF[f'y_ws{ws_id:02d}_z']
    r_  = rowF[f'y_ws{ws_id:02d}_roll']
    p_  = rowF[f'y_ws{ws_id:02d}_pitch']
    yw_ = rowF[f'y_ws{ws_id:02d}_yaw']

    # 轨道->全局
    X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(
        s_, s_vals, xvals, yvals, zvals, psi_vals, phi_vals
    )
    T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)

    # 轮对->轨道
    T_W2T = make_transform(yw_, p_, r_, 0.0, y_, z_)

    # 合成 -> 全局
    T_W2G = T_T2G @ T_W2T

    # axle
    pts_axle_G = transform_points_3d(T_W2G, pts_axle)
    ax2.plot_surface(
        pts_axle_G[:,0].reshape(X_axle.shape),
        pts_axle_G[:,1].reshape(X_axle.shape),
        pts_axle_G[:,2].reshape(X_axle.shape),
        color='magenta', alpha=1
    )
    pts_all_in_G.append(pts_axle_G)

    # left wheel
    pts_left_local = pts_wheel.copy()
    pts_left_local[:,1] += wheel_offset
    pts_left_G = transform_points_3d(T_W2G, pts_left_local)
    ax2.plot_surface(
        pts_left_G[:,0].reshape(X_wh.shape),
        pts_left_G[:,1].reshape(X_wh.shape),
        pts_left_G[:,2].reshape(X_wh.shape),
        color='cyan', alpha=1
    )
    pts_all_in_G.append(pts_left_G)

    # right wheel
    pts_right_local = pts_wheel.copy()
    pts_right_local[:,1] -= wheel_offset
    pts_right_G = transform_points_3d(T_W2G, pts_right_local)
    ax2.plot_surface(
        pts_right_G[:,0].reshape(X_wh.shape),
        pts_right_G[:,1].reshape(X_wh.shape),
        pts_right_G[:,2].reshape(X_wh.shape),
        color='cyan', alpha=1
    )
    pts_all_in_G.append(pts_right_G)

# ============ (2B) 2 转向架 ============
for bg_id in [1,2]:
    s_bg  = rowF[f'y_f0{bg_id}_x']
    y_bg  = rowF[f'y_f0{bg_id}_y']
    z_bg  = rowF[f'y_f0{bg_id}_z']
    r_bg  = rowF[f'y_f0{bg_id}_roll']
    p_bg  = rowF[f'y_f0{bg_id}_pitch']
    yw_bg = rowF[f'y_f0{bg_id}_yaw']

    X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(
        s_bg, s_vals, xvals, yvals, zvals, psi_vals, phi_vals
    )
    T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)
    T_bg2T = make_transform(yw_bg, p_bg, r_bg, 0.0, y_bg, z_bg)
    T_bg2G = T_T2G @ T_bg2T
    
    # 若不想单独网格，可以直接画
    draw_box(ax2, T_bg2G, bogie_box_faces, color='green', alpha=0.1)

    # 转向架 8个角点
    bogie_corners_local = np.array([
        [-1.5, -1.25, -0.2],
        [-1.5, -1.25, +0.2],
        [-1.5,  1.25, -0.2],
        [-1.5,  1.25, +0.2],
        [ 1.5, -1.25, -0.2],
        [ 1.5, -1.25, +0.2],
        [ 1.5,  1.25, -0.2],
        [ 1.5,  1.25, +0.2],
    ])
    bogie_corners_G = transform_points_3d(T_bg2G, bogie_corners_local)
    pts_all_in_G.append(bogie_corners_G)

# ============ (2C) 车体 ============
# 特殊规定: 车体几何高度 1.8
s_cb   = rowF['y_cb_x']
y_cb   = rowF['y_cb_y']
z_cb   = rowF['y_cb_z'] - 1.8 # hc = -1.8
r_cb   = rowF['y_cb_roll']
p_cb   = rowF['y_cb_pitch']
yw_cb  = rowF['y_cb_yaw']

X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(
    s_cb, s_vals, xvals, yvals, zvals, psi_vals, phi_vals
)
T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)
T_cb2T = make_transform(yw_cb, p_cb, r_cb, 0.0, y_cb, z_cb)
T_cb2G = T_T2G @ T_cb2T

# 车体需要翻转 180°
T_flipCar = make_transform(0, 0, math.pi, 0, 0, 0)
T_cb2G = T_cb2G @ T_flipCar

# 车体的8个顶点
car_body_corners_local = np.array([
    [-12.5, -1.5, -1.5],
    [-12.5, -1.5, +1.5],
    [-12.5,  1.5, -1.5],
    [-12.5,  1.5, +1.5],
    [ 12.5, -1.5, -1.5],
    [ 12.5, -1.5, +1.5],
    [ 12.5,  1.5, -1.5],
    [ 12.5,  1.5, +1.5],
])
car_body_corners_G = transform_points_3d(T_cb2G, car_body_corners_local)
pts_all_in_G.append(car_body_corners_G)

# 或直接画车体的网格
draw_box(ax2, T_cb2G, car_body_faces, color='orange', alpha=0.05)

# ============ (3) 合并并计算包围盒 ============
pts_all_in_G = np.vstack(pts_all_in_G)  # (N,3)
Xmin, Xmax = pts_all_in_G[:,0].min(), pts_all_in_G[:,0].max()
Ymin, Ymax = pts_all_in_G[:,1].min(), pts_all_in_G[:,1].max()
Zmin, Zmax = pts_all_in_G[:,2].min(), pts_all_in_G[:,2].max()

Xmid = 0.5*(Xmin + Xmax)
Ymid = 0.5*(Ymin + Ymax)
Zmid = 0.5*(Zmin + Zmax)
box_half_size = 0.5 * max(Xmax - Xmin, Ymax - Ymin, Zmax - Zmin)
box_half_size += 0.5  # 多留点余量

ax2.set_xlim(Xmid - box_half_size, Xmid + box_half_size)
ax2.set_ylim(Ymid - box_half_size, Ymid + box_half_size)
ax2.set_zlim(Zmid - box_half_size, Zmid + box_half_size)

ax2.set_xlabel("X [m]")
ax2.set_ylabel("Y [m]")
ax2.set_zlabel("Z [m]")
ax2.set_title("Close-up of Whole Train (Last Frame)")
ax2.invert_zaxis()  # 如果Z越大要越低
ax2.set_box_aspect((1,1,1))
ax2.view_init(elev=30, azim=-60, roll=0)
ax2.legend()

plt.tight_layout()
plt.show()

# 制作动画
- **生成**列车在轨道上的整体运动 GlobalView_Vehicle.mp4
- **生成**列车特写镜头 CloseupView_Vehicle.mp4
- 生成 200 帧视频约用时 3.5 min

In [ ]:
%matplotlib widget   

import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import json

# ------------- 导入工具函数 -------------
from tools_RailTransform import (
    # 轨道姿态
    get_track_pose,
    # 坐标变换
    make_transform,
    transform_points_3d,
    # 几何网格
    create_cylinder_mesh,
    create_box_mesh,
    # 绘图辅助
    draw_wheelset,
    draw_box,
    # 读取 SIMPACK
    read_simpack_6dof,
    # (可选) 计算包围盒
    compute_points_bounding_box
)

# =========================== 1) 轨道中心线数据 ===========================
with open('trajectory_data.json', 'r') as jf:
    trajectory_data = json.load(jf)

s_vals   = np.array(trajectory_data['s'])
xvals    = np.array(trajectory_data['x'])
yvals    = np.array(trajectory_data['y'])
zvals    = np.array(trajectory_data['z'])
psi_vals = np.array(trajectory_data['psi'])
phi_vals = np.array(trajectory_data['phi'])

left_rail  = np.array(trajectory_data['left_rail'])   # (N,3) 左轨
right_rail = np.array(trajectory_data['right_rail'])  # (N,3) 右轨

print(f"轨道数据加载完成，数量={len(s_vals)}")

# =========================== 2) 读取SIMPACK输出 ===========================
file_path = os.path.join('..', 'PostAnalysis', 'Result_Y_RosRt.log')
df_wheel = read_simpack_6dof(file_path)
print(f"SIMPACK日志加载完成: {df_wheel.shape}")

# =========================== 3) 生成几何模型 (车轴/车轮/车体/转向架) ===========================
axle_length   = 2.0
axle_radius   = 0.065
wheel_radius  = 0.43
wheel_thickness = 0.04
wheel_offset  = 0.7175

X_axle, Y_axle, Z_axle = create_cylinder_mesh(axle_length, axle_radius)
pts_axle = np.column_stack([X_axle.ravel(), Y_axle.ravel(), Z_axle.ravel()])

X_wh, Y_wh, Z_wh = create_cylinder_mesh(wheel_thickness, wheel_radius)
pts_wheel = np.column_stack([X_wh.ravel(), Y_wh.ravel(), Z_wh.ravel()])

car_body_faces  = create_box_mesh(L=25.0, W=3.0, H=3.0, nx=2, ny=2, nz=2)
bogie_box_faces = create_box_mesh(L=3.0,  W=2.5, H=0.4, nx=2, ny=2, nz=2)

# =========================== 4) 几个绘制函数(针对不同视角) ===========================
def plot_global_frame(ax, df_wheel, row_idx):
    """
    以全局视角绘制列车各部件 + 轨道
    """
    ax.cla()  # 清空上一次画面

    # (A) 画轨道线
    ax.plot(xvals, yvals, zvals, color='black', alpha=0.4, label='Track Center')
    ax.plot(left_rail[:,0], left_rail[:,1], left_rail[:,2], 'r-', alpha=0.4, label='Left Rail')
    ax.plot(right_rail[:,0], right_rail[:,1], right_rail[:,2], 'b-', alpha=0.4, label='Right Rail')

    # (B) 获取当前帧数据
    row = df_wheel.iloc[row_idx]

    # (B1) 轮对
    for ws_id in [1,2,3,4]:
        s_  = row[f'y_ws{ws_id:02d}_x']
        y_  = row[f'y_ws{ws_id:02d}_y']
        z_  = row[f'y_ws{ws_id:02d}_z']
        r_  = row[f'y_ws{ws_id:02d}_roll']
        p_  = row[f'y_ws{ws_id:02d}_pitch']
        yw_ = row[f'y_ws{ws_id:02d}_yaw']

        X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(
            s_, s_vals, xvals, yvals, zvals, psi_vals, phi_vals
        )
        T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)
        T_W2T = make_transform(yw_, p_, r_, 0.0, y_, z_)
        T_W2G = T_T2G @ T_W2T

        draw_wheelset(ax, T_W2G, 
                      pts_axle, X_axle.shape,
                      pts_wheel, X_wh.shape,
                      wheel_offset)

    # (B2) 转向架
    for bg_id in [1,2]:
        s_bg  = row[f'y_f0{bg_id}_x']
        y_bg  = row[f'y_f0{bg_id}_y']
        z_bg  = row[f'y_f0{bg_id}_z']
        r_bg  = row[f'y_f0{bg_id}_roll']
        p_bg  = row[f'y_f0{bg_id}_pitch']
        yw_bg = row[f'y_f0{bg_id}_yaw']

        X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(
            s_bg, s_vals, xvals, yvals, zvals, psi_vals, phi_vals
        )
        T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)
        T_bg2T = make_transform(yw_bg, p_bg, r_bg, 0.0, y_bg, z_bg)
        T_bg2G = T_T2G @ T_bg2T

        draw_box(ax, T_bg2G, bogie_box_faces, color='green', alpha=0.1)

    # (B3) 车体几何高 1.8 m, 并需要翻转 180°
    s_cb   = row['y_cb_x']
    y_cb   = row['y_cb_y']
    z_cb   = row['y_cb_z'] - 1.8 # hc = -1.8
    r_cb   = row['y_cb_roll']
    p_cb   = row['y_cb_pitch']
    yw_cb  = row['y_cb_yaw']

    X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(
        s_cb, s_vals, xvals, yvals, zvals, psi_vals, phi_vals
    )
    T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)
    T_cb2T = make_transform(yw_cb, p_cb, r_cb, 0.0, y_cb, z_cb)
    T_cb2G = T_T2G @ T_cb2T

    T_flipCar = make_transform(0, 0, math.pi, 0, 0, 0) # 车体翻转
    T_cb2G = T_cb2G @ T_flipCar

    draw_box(ax, T_cb2G, car_body_faces, color='orange', alpha=0.05)

    # (C) 坐标与视图
    ax.set_xlabel("X [m]")
    ax.set_ylabel("Y [m]")
    ax.set_zlabel("Z [m]")
    ax.set_title(f"Global View - Frame {row_idx}")
    ax.legend()

    # 强制让坐标范围包住轨道全长
    x_min, x_max = xvals.min(), xvals.max()
    y_min, y_max = yvals.min(), yvals.max()
    z_min, z_max = zvals.min(), zvals.max()
    cx = 0.5*(x_min + x_max)
    cy = 0.5*(y_min + y_max)
    cz = 0.5*(z_min + z_max)
    max_range = max(x_max - x_min, y_max - y_min, z_max - z_min)*0.5
    ax.set_xlim(cx - max_range, cx + max_range)
    ax.set_ylim(cy - max_range, cy + max_range)
    ax.set_zlim(cz - max_range, cz + max_range)
    ax.set_box_aspect((1,1,1))

    # 翻转Z轴，使Z越大图中越“低”
    ax.invert_zaxis()
    ax.view_init(elev=25, azim=70, roll=0)

def plot_closeup_frame(ax, df_wheel, row_idx):
    """
    特写视角：更紧凑的范围只包住当前车体+轮对+转向架。
    """
    ax.cla()

    # 可选：依然绘制轨道线(若嫌碍眼可不绘制)
    ax.plot(xvals, yvals, zvals, color='black', alpha=0.8, label='Track Centerline')
    ax.plot(left_rail[:,0], left_rail[:,1], left_rail[:,2], 'r-', alpha=0.8, label='Left Rail')
    ax.plot(right_rail[:,0], right_rail[:,1], right_rail[:,2], 'b-', alpha=0.8, label='Right Rail')

    row = df_wheel.iloc[row_idx]
    pts_all_in_G = []  # 收集所有部件的全局点，用于自动计算包围盒

    # (A) 4 轮对
    for ws_id in [1,2,3,4]:
        s_  = row[f'y_ws{ws_id:02d}_x']
        y_  = row[f'y_ws{ws_id:02d}_y']
        z_  = row[f'y_ws{ws_id:02d}_z']
        r_  = row[f'y_ws{ws_id:02d}_roll']
        p_  = row[f'y_ws{ws_id:02d}_pitch']
        yw_ = row[f'y_ws{ws_id:02d}_yaw']

        X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(
            s_, s_vals, xvals, yvals, zvals, psi_vals, phi_vals
        )
        T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)
        T_W2T = make_transform(yw_, p_, r_, 0.0, y_, z_)
        T_W2G = T_T2G @ T_W2T

        # 车轴
        pts_axle_G = transform_points_3d(T_W2G, pts_axle)
        ax.plot_surface(
            pts_axle_G[:,0].reshape(X_axle.shape),
            pts_axle_G[:,1].reshape(X_axle.shape),
            pts_axle_G[:,2].reshape(X_axle.shape),
            color='magenta', alpha=1
        )
        pts_all_in_G.append(pts_axle_G)

        # 左轮
        pts_left_local = pts_wheel.copy()
        pts_left_local[:,1] += wheel_offset
        pts_left_G = transform_points_3d(T_W2G, pts_left_local)
        ax.plot_surface(
            pts_left_G[:,0].reshape(X_wh.shape),
            pts_left_G[:,1].reshape(X_wh.shape),
            pts_left_G[:,2].reshape(X_wh.shape),
            color='cyan', alpha=1
        )
        pts_all_in_G.append(pts_left_G)

        # 右轮
        pts_right_local = pts_wheel.copy()
        pts_right_local[:,1] -= wheel_offset
        pts_right_G = transform_points_3d(T_W2G, pts_right_local)
        ax.plot_surface(
            pts_right_G[:,0].reshape(X_wh.shape),
            pts_right_G[:,1].reshape(X_wh.shape),
            pts_right_G[:,2].reshape(X_wh.shape),
            color='cyan', alpha=1
        )
        pts_all_in_G.append(pts_right_G)

    # (B) 2 转向架
    for bg_id in [1,2]:
        s_bg  = row[f'y_f0{bg_id}_x']
        y_bg  = row[f'y_f0{bg_id}_y']
        z_bg  = row[f'y_f0{bg_id}_z']
        r_bg  = row[f'y_f0{bg_id}_roll']
        p_bg  = row[f'y_f0{bg_id}_pitch']
        yw_bg = row[f'y_f0{bg_id}_yaw']

        X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(
            s_bg, s_vals, xvals, yvals, zvals, psi_vals, phi_vals
        )
        T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)
        T_bg2T = make_transform(yw_bg, p_bg, r_bg, 0.0, y_bg, z_bg)
        T_bg2G = T_T2G @ T_bg2T

        draw_box(ax, T_bg2G, bogie_box_faces, color='green', alpha=0.1)

        # 转向架8个角点
        bogie_corners_local = np.array([
            [-1.5, -1.25, -0.2],
            [-1.5, -1.25, +0.2],
            [-1.5,  1.25, -0.2],
            [-1.5,  1.25, +0.2],
            [ 1.5, -1.25, -0.2],
            [ 1.5, -1.25, +0.2],
            [ 1.5,  1.25, -0.2],
            [ 1.5,  1.25, +0.2],
        ])
        bogie_corners_G = transform_points_3d(T_bg2G, bogie_corners_local)
        pts_all_in_G.append(bogie_corners_G)

    # (C) 车体
    s_cb   = row['y_cb_x']
    y_cb   = row['y_cb_y']
    z_cb   = row['y_cb_z'] - 1.8 # hc = -1.8
    r_cb   = row['y_cb_roll']
    p_cb   = row['y_cb_pitch']
    yw_cb  = row['y_cb_yaw']

    X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(
        s_cb, s_vals, xvals, yvals, zvals, psi_vals, phi_vals
    )
    T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)
    T_cb2T = make_transform(yw_cb, p_cb, r_cb, 0.0, y_cb, z_cb)
    T_cb2G = T_T2G @ T_cb2T

    T_flipCar = make_transform(0, 0, math.pi, 0, 0, 0)
    T_cb2G = T_cb2G @ T_flipCar

    draw_box(ax, T_cb2G, car_body_faces, color='orange', alpha=0.05)

    # 车体8个角点
    car_body_corners_local = np.array([
        [-12.5, -1.5, -1.5],
        [-12.5, -1.5, +1.5],
        [-12.5,  1.5, -1.5],
        [-12.5,  1.5, +1.5],
        [ 12.5, -1.5, -1.5],
        [ 12.5, -1.5, +1.5],
        [ 12.5,  1.5, -1.5],
        [ 12.5,  1.5, +1.5],
    ])
    car_body_corners_G = transform_points_3d(T_cb2G, car_body_corners_local)
    pts_all_in_G.append(car_body_corners_G)

    # (D) 根据全部顶点，自动设定可视范围 (若使用了 compute_points_bounding_box)
    (xlim, Xmax), (ylim, Ymax), (zlim, Zmax) = compute_points_bounding_box(pts_all_in_G, extra_margin=0.5)
    ax.set_xlim(xlim, Xmax)
    ax.set_ylim(ylim, Ymax)
    ax.set_zlim(zlim, Zmax)

    ax.set_xlabel("X [m]")
    ax.set_ylabel("Y [m]")
    ax.set_zlabel("Z [m]")
    ax.set_title(f"Close-up - Frame {row_idx}")
    ax.invert_zaxis()
    ax.set_box_aspect((1,1,1))
    ax.view_init(elev=30, azim=-60, roll=0)
    ax.legend()

# =========================== 5) 生成动画 ===========================
num_frames = 200
total_rows = len(df_wheel)
step = total_rows // num_frames
frame_indices = range(0, total_rows, step)

# (A) 全局视角动画
fig_global = plt.figure(figsize=(12,6))
ax_global = fig_global.add_subplot(111, projection='3d')

def init_func_global():
    return []

def update_func_global(frame):
    row_idx = frame_indices[frame]
    plot_global_frame(ax_global, df_wheel, row_idx)
    return []

anim_global = FuncAnimation(
    fig_global, update_func_global,
    frames=len(frame_indices),
    init_func=init_func_global,
    interval=100,  # 毫秒
    blit=False
)

anim_global.save('GlobalView_Vehicle.mp4', fps=10, dpi=100)

# (B) 特写视角动画
fig_closeup = plt.figure(figsize=(8,6))
ax_closeup = fig_closeup.add_subplot(111, projection='3d')

def init_func_closeup():
    return []

def update_func_closeup(frame):
    row_idx = frame_indices[frame]
    plot_closeup_frame(ax_closeup, df_wheel, row_idx)
    return []

anim_closeup = FuncAnimation(
    fig_closeup, update_func_closeup,
    frames=len(frame_indices),
    init_func=init_func_closeup,
    interval=100,
    blit=False
)

anim_closeup.save('CloseupView_Vehicle.mp4', fps=10, dpi=100)

plt.show()
